## This is the test version

In [ ]:
# -- coding: utf-8 --
"""
Created on Mon May 12 22:44:27 2025

"""

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np

dataset = pd.read_csv('predict_bonus.csv', sep=';')

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

#print(X)
#print(y)

#interpolation of missing data
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 7:8])
X[:, 7:8] = np.round(imputer.transform(X[:, 7:8]), 2)

#encoding of features (betriebliche Bewertung)
#ct = ColumnTransformer(
#    transformers=[('encoder', OneHotEncoder(drop='first'), list(range(7)))],
#   remainder='passthrough')
# Mapping-Dictionary
bewertung_map = {
    'uebertroffen': 6,
    'erfuellt': 5,
    'knapp erfuellt': 4,
    'nicht erfuellt': 3
}

# Funktion, die das Mapping auf mehrere Spalten anwendet
def map_bewertung(X):
    return np.vectorize(bewertung_map.get)(X)

# Transformer, der das Mapping durchführt
mapping_transformer = FunctionTransformer(map_bewertung)

# ColumnTransformer mit Mapping für die ersten 7 Spalten
ct = ColumnTransformer(
    transformers=[('mapper', mapping_transformer, list(range(7)))],
    remainder='passthrough'
)

# Transformation anwenden
X = np.array(ct.fit_transform(X))

#print('Matrix of features: \n' + str(X))

#validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#print(X_train)
#print(X_test)
#print(y_train)
#print(y_test)

#feature scaling
startIndex = 1
sc = StandardScaler()
X_train[:, startIndex:] = sc.fit_transform(X_train[:, startIndex:])
X_test[:, startIndex:] = sc.transform(X_test[:, startIndex:])

X_train = X_train.astype(np.float64)
X_test  = X_test .astype(np.float64)

np.set_printoptions(precision=2, suppress=True)

#print(X_train)
#print(X_test)

#normalization

#multiple linear regression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

#predication
y_pred = regressor.predict(X_test)

print(
    np.concatenate(
        (y_pred.reshape(len(y_pred), 1),
         y_test.reshape(len(y_test ), 1)),
        axis=1
    )
)